In [2]:
import tensorflow as tf
from keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [6]:
# Configuration
IMG_SIZE = (256, 256)  # InceptionV3 input size
BATCH_SIZE = 32

EPOCHS = 12
LEARNING_RATE = 0.0001
TRAIN_DIR = r"C:\Users\Public\Documents\watermelon"# Replace with the path to your training data
VAL_DIR = r"C:\Users\Public\Documents\watermelon"# Replace with the path to your validation data
NUM_CLASSES = 4 # Replace with the number of classes in your dataset

In [7]:
# Load the InceptionV3 model pre-trained on ImageNet, excluding the top layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
print(base_model.summary())

Model: "inception_v3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)    │ (None, 256, 256, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_94 (Conv2D)            │ (None, 127, 127, 32)      │             864 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_94        │ (None, 127, 127, 32)      │              96 │ conv2d_94[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_94 (Activation)    │ (None, 127, 127, 32)      │               0 │ batch_normalization_94[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_95 (Conv2D)            │ (None, 125, 125, 32)      │           9,216 │ activation_94[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_95        │ (None, 125, 125, 32)      │              96 │ conv2d_95[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_95 (Activation)    │ (None, 125, 125, 32)      │               0 │ batch_normalization_95[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_96 (Conv2D)            │ (None, 125, 125, 64)      │          18,432 │ activation_95[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_96        │ (None, 125, 125, 64)      │             192 │ conv2d_96[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_96 (Activation)    │ (None, 125, 125, 64)      │               0 │ batch_normalization_96[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_4               │ (None, 62, 62, 64)        │               0 │ activation_96[0][0]        │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_97 (Conv2D)            │ (None, 62, 62, 80)        │           5,120 │ max_pooling2d_4[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_97        │ (None, 62, 62, 80)        │             240 │ conv2d_97[0][0]            │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_97 (Activation)    │ (None, 62, 62, 80)        │               0 │ batch_normalization_97[0]… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_98 (Conv2D)            │ (None, 60, 60, 192)       │         138,24

 Total params: 21,802,784 (83.17 MB)

 Trainable params: 21,768,352 (83.04 MB)

 Non-trainable params: 34,432 (134.50 KB)

None


In [8]:
# Freeze the base model layers to use for feature extraction

for layer in base_model.layers[:-50]:
    layer.trainable = False
for layer in base_model.layers[-50:]:
    layer.trainable = True

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global Average Pooling
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)   # Fully connected layer
predictions = Dense(NUM_CLASSES, activation='softmax')(x)  # Output laye

In [9]:
# Combine base model and custom layers
model = Model(inputs=base_model.input, outputs=predictions)

In [10]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
# Data generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0/255.0)



In [12]:
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 120 images belonging to 4 classes.
Found 120 images belonging to 4 classes.


In [17]:
# Callbacks for saving the best model and early stopping
checkpoint = ModelCheckpoint(
    'watermelon_model.keras',  # Correct extension for full model saving
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)





In [18]:
early_stopping = EarlyStopping(monitor='val_accuracy',
                               patience=5,
                               restore_best_weights=True)


In [19]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[checkpoint, early_stopping]
)

Epoch 1/12


C:\Users\ELANGO S\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3/3 ━━━━━━━━━━━━━━━━━━━━ 81s 18s/step - accuracy: 0.4076 - loss: 1.3552 - val_accuracy: 0.7292 - val_loss: 0.9470
Epoch 2/12
1/3 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.9167 - loss: 0.9716

C:\Users\ELANGO S\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.9167 - loss: 0.9716 - val_accuracy: 0.6667 - val_loss: 0.9405
Epoch 3/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 40s 11s/step - accuracy: 0.9025 - loss: 0.8054 - val_accuracy: 0.9688 - val_loss: 0.4685
Epoch 4/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.9688 - loss: 0.6585 - val_accuracy: 1.0000 - val_loss: 0.4078
Epoch 5/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 35s 9s/step - accuracy: 1.0000 - loss: 0.4143 - val_accuracy: 0.9688 - val_loss: 0.2241
Epoch 6/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.9688 - loss: 0.2725 - val_accuracy: 1.0000 - val_loss: 0.1230
Epoch 7/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 36s 9s/step - accuracy: 0.9899 - loss: 0.2121 - val_accuracy: 0.9896 - val_loss: 0.0740
Epoch 8/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 1.0000 - loss: 0.1068 - val_accuracy: 1.0000 - val_loss: 0.0393
Epoch 9/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 35s 9s/step - accuracy: 1.0000 - loss: 0.0787 - val_accuracy: 1.0000 - val_loss: 0.0176


In [20]:
# Evaluate the model
loss, accuracy = model.evaluate(val_generator)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

# Unfreeze the base model for fine-tuning
base_model.trainable = True

# Recompile the model with a lower learning rate for fine-tuning
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss='categorical_crossentropy',

              metrics=['accuracy'])

# Fine-tune the mode+l


4/4 ━━━━━━━━━━━━━━━━━━━━ 18s 4s/step - accuracy: 0.9881 - loss: 0.3847
Validation Loss: 0.3893386721611023
Validation Accuracy: 0.9833333492279053


In [21]:
history_fine_tune = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[checkpoint, early_stopping]
)



Epoch 1/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 76s 16s/step - accuracy: 0.9943 - loss: 0.4007 - val_accuracy: 0.9896 - val_loss: 0.1745
Epoch 2/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.9688 - loss: 0.2393 - val_accuracy: 1.0000 - val_loss: 0.0815
Epoch 3/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 37s 9s/step - accuracy: 1.0000 - loss: 0.1660 - val_accuracy: 1.0000 - val_loss: 0.0376
Epoch 4/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.9688 - loss: 0.1270 - val_accuracy: 1.0000 - val_loss: 0.0147
Epoch 5/12
3/3 ━━━━━━━━━━━━━━━━━━━━ 36s 9s/step - accuracy: 1.0000 - loss: 0.0500 - val_accuracy: 1.0000 - val_loss: 0.0045


In [29]:
model.save("watermelon_model_new.keras")
# # Final evaluation
loss, accuracy = model.evaluate(val_generator)
print(f"Final Validation Loss: {loss}")
print(f"Final Validation Accuracy: {accuracy}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 27s 4s/step - accuracy: 0.9852 - loss: 0.1658
Final Validation Loss: 0.16475872695446014
Final Validation Accuracy: 0.9916666746139526


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
import numpy as np
from PIL import Image




In [3]:
# Load the trained Inception V3 model
model = tf.keras.models.load_model("watermelon_model.keras")  # Replace with your model path
#model.summary()


       

In [4]:

def preprocess_image(image_path):
    img = Image.open(image_path).resize((256, 256))  # Resize to 299x299
    img_array = image.img_to_array(img)  # Convert image to array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = preprocess_input(img_array)  # Preprocess image for InceptionV3
    return img_array

In [5]:
# Define the mapping of class indices to disease names
class_names = {
    0: "Anthracnose",
    1: "Downy_Mildew",
    2: "Healthy",
    3: "Mosaic_Virus",
    # Add more classes as needed
}

In [6]:
def predict_disease(image_path, threshold=0.8, top_n_classes=3):
    # Preprocess the input image
    input_image = preprocess_image(image_path)
    
    # Get predictions from the model
    predictions = model.predict(input_image)
    
    # Calculate softmax probabilities and get the max confidence
    probabilities = tf.nn.softmax(predictions[0])  # Apply softmax to get probabilities
    max_confidence = np.max(probabilities)  # Get the max confidence
    predicted_class = np.argmax(probabilities)  # Get the predicted class index
    
    # Print the confidence scores for debugging
    print(f"Predicted Class: {predicted_class}, Max Confidence: {max_confidence}")
    
    # Get the top N classes with their probabilities
    top_n_classes_indices = tf.argsort(probabilities, direction='DESCENDING')[:top_n_classes]
    top_n_probs = tf.gather(probabilities, top_n_classes_indices)
    
    # Print the top N class probabilities for debugging
    print(f"Top {top_n_classes} Class Indices: {top_n_classes_indices.numpy()}")
    print(f"Top {top_n_classes} Class Probabilities: {top_n_probs.numpy()}")
    
   # Check for OOD (confidence below the threshold)
    if any(prob > threshold for prob in top_n_probs.numpy()):
        if predicted_class in class_names:
            disease_name = class_names[predicted_class]
            return f"Disease Class: {disease_name}, Confidence: {max_confidence:.2f}"
        else:
            return f"Unknown Class: {predicted_class}, Confidence: {max_confidence:.2f}"
    else:
        return "Irrelevant Image"

    # if max_confidence < threshold:
    #     return "Out-of-Distribution (Irrelevant Image)"
    # else:
    #     disease_name = class_names.get(predicted_class, "Unknown Disease")
    #     return f"Disease Class: {disease_name}, Confidence: {max_confidence:.2f}"


In [30]:
# irrelevent image
image_path = r"C:\Users\ELANGO S\OneDrive\Pictures\Saved Pictures\123.webp"  # Replace with the new image path
result = predict_disease(image_path, threshold=0.3)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
Predicted Class: 3, Max Confidence: 0.2934718430042267
Top 3 Class Indices: [3 2 1]
Top 3 Class Probabilities: [0.29347184 0.24880567 0.23235504]
Irrelevant Image


In [27]:
# relevent image
image_path = r"C:\Users\Public\Documents\watermelon\Mosaic_Virus\IMG_3074_shifted.jpg" # Replace with the new image pat
result = predict_disease(image_path, threshold=0.3)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
Predicted Class: 3, Max Confidence: 0.35363903641700745
Top 3 Class Indices: [3 2 1]
Top 3 Class Probabilities: [0.35363904 0.2282969  0.2201871 ]
Disease Class: Mosaic_Virus, Confidence: 0.35


In [28]:
# relevent image
image_path = r"C:\Users\Public\Documents\watermelon\Anthracnose\IMG_4217_scaled.jpg" # Replace with the new image pat
result = predict_disease(image_path, threshold=0.3)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
Predicted Class: 0, Max Confidence: 0.39490967988967896
Top 3 Class Indices: [0 3 1]
Top 3 Class Probabilities: [0.39490968 0.2072531  0.20155674]
Disease Class: Anthracnose, Confidence: 0.39


In [29]:
# relevent image
image_path = r"C:\Users\Public\Documents\watermelon\Downy_Mildew\IMG_3359_brightened.jpg" # Replace with the new image pat
result = predict_disease(image_path, threshold=0.3)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
Predicted Class: 1, Max Confidence: 0.34879234433174133
Top 3 Class Indices: [1 2 3]
Top 3 Class Probabilities: [0.34879234 0.23818055 0.21427934]
Disease Class: Downy_Mildew, Confidence: 0.35


In [31]:

# relevent image
image_path = r"C:\Users\Public\Documents\watermelon\Healthy\IMG_2907_zoomed.jpg" # Replace with the new image pat
result = predict_disease(image_path, threshold=0.3)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
Predicted Class: 2, Max Confidence: 0.34281814098358154
Top 3 Class Indices: [2 1 3]
Top 3 Class Probabilities: [0.34281814 0.23188221 0.22968517]
Disease Class: Healthy, Confidence: 0.34


In [7]:
# relevent image
image_path = r"C:\Users\Public\Documents\watermelon\Anthracnose\IMG_4217_scaled.jpg" # Replace with the new image pat
result = predict_disease(image_path, threshold=0.3)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Predicted Class: 0, Max Confidence: 0.39490967988967896
Top 3 Class Indices: [0 3 1]
Top 3 Class Probabilities: [0.39490968 0.2072531  0.20155674]
Disease Class: Anthracnose, Confidence: 0.39


In [9]:
# relevent image
image_path = r"C:\Users\ELANGO S\OneDrive\Pictures\Saved Pictures\123.webp" # Replace with the new image pat
result = predict_disease(image_path, threshold=0.3)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
Predicted Class: 3, Max Confidence: 0.2934718430042267
Top 3 Class Indices: [3 2 1]
Top 3 Class Probabilities: [0.29347184 0.24880567 0.23235504]
Irrelevant Image


In [10]:
# relevent image
image_path = r"C:\Users\ELANGO S\OneDrive\Pictures\Saved Pictures\123.webp" # Replace with the new image pat
result = predict_disease(image_path, threshold=0.3)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
Predicted Class: 3, Max Confidence: 0.2934718430042267
Top 3 Class Indices: [3 2 1]
Top 3 Class Probabilities: [0.29347184 0.24880567 0.23235504]
Irrelevant Image
